In [ ]:
#### i checked some code section in original json file, needed to be modified as they are not precise ! 
##  in the following code we are modifying the code section based on new start and end lines in the new sol files

In [27]:
import os
import json
from tqdm import tqdm

# === Configuration ===
dataset_name = "TD_FTSmartAudit_datasets"
raw_dir = "../../data/dataset/raw"
json_file_path = os.path.join(raw_dir, f"{dataset_name}.json")
# New base directory for final processed data
base_output_dir = f"../../data/processed_data/{dataset_name}/final/"

# Define the vulnerability type we want to process.
desired_vuln_types = "time_manipulation"
vuln = desired_vuln_types

# New directories for saving processed contracts and LOC info.
contracts_dir = os.path.join(base_output_dir, vuln, "contracts")
new_locs_dir = os.path.join(base_output_dir, vuln, "LOCs")
os.makedirs(contracts_dir, exist_ok=True)
os.makedirs(new_locs_dir, exist_ok=True)

# Existing LOCs are stored in a different directory.
existing_locs_dir = os.path.join("../../data/processed_data", dataset_name, "LOCs")

# --- Load dataset ---
with open(json_file_path, "r", encoding="utf-8") as file:
    data = json.load(file)

print(f"Total contracts in dataset: {len(data)}")
end_contract = len(data)

for idx, record in tqdm(enumerate(data[:end_contract]), total=end_contract, desc="Processing Contracts"):
    vuln_type = record.get("Vulnerability Type", "").strip().lower()
    contracts_dir = os.path.join(base_output_dir, vuln_type, "contracts")
    new_locs_dir = os.path.join(base_output_dir, vuln_type, "LOCs")
    os.makedirs(contracts_dir, exist_ok=True)
    os.makedirs(new_locs_dir, exist_ok=True)

    source_code_full = record.get("Source Code", "")
    input_code = record.get("input", "")
    
    if not input_code:
        print(f"Skipping contract {idx}: No input found.")
        continue

    contract_filename = f"{idx}.sol"
    contract_path = os.path.join(contracts_dir, contract_filename)
    with open(contract_path, "w", encoding="utf-8") as f:
        f.write(input_code)

    source_lines = source_code_full.split("\n")
    input_lines_list = input_code.split("\n")
    
    # Compute the bias as the number of lines before the first "pragma" statement
    bias = 0
    for i, line in enumerate(source_lines):
        if "pragma" in line:
            bias = i
            break
    
    existing_vuln_json_path = os.path.join(existing_locs_dir, f"{idx}.json")
    if not os.path.exists(existing_vuln_json_path):
        print(f"Skipping contract {idx}: Vulnerability LOC file not found in existing LOCs folder.")
        continue

    with open(existing_vuln_json_path, "r", encoding="utf-8") as json_file:
        vulnerability_info = json.load(json_file)
    
    for block in vulnerability_info:
        original_start = block.get("start_line")
        original_end = block.get("end_line")
        if original_start is None or original_end is None:
            continue
        
        new_start = original_start - bias
        new_end = original_end - bias
        
        block["start_line"] = new_start
        block["end_line"] = new_end
        block["code"] = input_lines_list[new_start - 1:new_end]
    
    new_vuln_json_path = os.path.join(new_locs_dir, f"{idx}.json")
    with open(new_vuln_json_path, "w", encoding="utf-8") as json_file:
        json.dump(vulnerability_info, json_file, indent=4)

    print(f"Processed contract {idx} ({vuln_type}): Updated LOC info with bias {bias}.")

print("Processing complete!")


Total contracts in dataset: 5


Processing Contracts: 100%|██████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 500.00it/s]

Processed contract 0 (time_manipulation): Updated LOC info with bias 7.
Processed contract 1 (time_manipulation): Updated LOC info with bias 6.
Processed contract 2 (time_manipulation): Updated LOC info with bias 6.
Processed contract 3 (time_manipulation): Updated LOC info with bias 6.
Processed contract 4 (time_manipulation): Updated LOC info with bias 6.
Processing complete!
